# New linkage generation

In [1]:
from generate_module import base, create_revolute_joint

from util import *
from timor.Geometry import Mesh
import os
from timor.Bodies import Body, Connector, Gender
from timor.utilities.transformation import Transformation
from timor.utilities.spatial import rotX, rotY, rotZ
from timor.Module import AtomicModule, ModulesDB, ModuleHeader
from timor.Joints import Joint
from timor.Geometry import Box, ComposedGeometry, Cylinder, Sphere, Mesh
from timor.Module import *

2025-04-12 23:43:40,34 Timor INFO Loading custom configurations from /home/jonas/.config/timor.config
2025-04-12 23:43:41,349 Timor INFO Getting robot modrob-gen2.
2025-04-12 23:43:41,351 Timor WARNING Could not fetch robot modrob-gen2 due to [Errno 17] File exists: '/home/jonas/miniconda3/envs/timor-env/lib/python3.10/site-packages/cache/robots/modrob-gen2'.


In [2]:
def create_connectors_for_link(length, diameter, trans):
    connectors = []
    for i in range(2):
        connectors.append(Connector(
            f'{int(diameter * 100)}-{i}', 
            trans[i],
            gender=Gender.f if i == 0 else Gender.m,
            connector_type='default', 
            size=[diameter, diameter]))
    return connectors

def create_i_links(rod_name) -> ModulesDB:
    """For every size, creates an I-shaped link (aka a cylinder) with two connectors."""
    ROT_X = Transformation.from_rotation(rotX(-np.pi/2)[:3, :3])
    sizes = (150 / 1000, 300 / 1000, 450 / 1000)
    diameter = 25 / 1000
    links = ModulesDB()
    
    
    for size in sizes:
        rod_id = f'{rod_name}-{size}'
        trans = read_rod_trans(rod_name, size, diameter)
        header = generate_header(rod_id, rod_id)
        connectors = create_connectors_for_link(size, diameter, trans)
        geometry = Box({'x': diameter, 'y': diameter, 'z': size}, pose=Transformation.from_translation([0, 0, 0]))
        body = Body(rod_id, collision=geometry, connectors=connectors,
                    inertia=square_rod_inertia(size, diameter))
        links.add(AtomicModule(header, [body]))
    return links

In [103]:
def read_rod_trans(rod_name, length, diameter):
    # Rot
    ROT_X_Reverse_180 = Transformation.from_rotation(rotX(-np.pi)[:3, :3])
    ROT_X_Reverse_90  = Transformation.from_rotation(rotX(-np.pi/2)[:3, :3])
    ROT_X_90  = Transformation.from_rotation(rotX(np.pi/2)[:3, :3])
    ROT_Y_Reverse_90  = Transformation.from_rotation(rotY(-np.pi/2)[:3, :3])
    ROT_Y_90  = Transformation.from_rotation(rotY(np.pi/2)[:3, :3])
    ROT_Y_Reverse_180 = Transformation.from_rotation(rotY(-np.pi)[:3, :3])


    # 540
    config_540_out_1 = ROT_Y_Reverse_90 @ Transformation.from_translation([-length/2+diameter/2, -length/2+0.25*diameter, 1.25*diameter])
    config_540_out_2 = ROT_X_90 @ Transformation.from_translation([0, -length+1.25*diameter/2, 1.25*diameter])
    config_540_in_1 = ROT_X_Reverse_90 @ Transformation.from_translation([0, -length/2+diameter/2, -1.5*diameter])
    config_540_in_2 = ROT_X_Reverse_180 @ Transformation.from_translation([0, 0, -length+2*diameter])
    
    # 430
    config_430_out_1 = ROT_Y_Reverse_90 @ Transformation.from_translation([-length/2+diameter/2, 0, diameter])
    config_430_out_2 = ROT_X_Reverse_180 @ Transformation.from_translation([0, 0, length/2+0.5*diameter])
    config_430_in_1 = ROT_X_Reverse_90 @ Transformation.from_translation([0, -length/2+1.25*diameter, -2.25*diameter])
    config_430_in_2 = ROT_X_Reverse_180 @ Transformation.from_translation([0, 0.75*diameter, -length+1.25*diameter])
    
    # 330
    config_330_out_1 = ROT_Y_Reverse_180 @ Transformation.from_translation([0, 0, length/2+diameter/2])
    config_330_out_2 = ROT_X_90 @ Transformation.from_translation([0, -length/2+diameter/2, diameter])
    config_330_in_1 = ROT_X_Reverse_180 @ Transformation.from_translation([0, 0, -length/2+diameter])
    config_330_in_2 = ROT_Y_90 @ ROT_X_Reverse_180 @ Transformation.from_translation([-length/2+diameter/2, 0, -diameter/2])
    
    
    # base
    base_out = ROT_Y_Reverse_90 @ Transformation.from_translation([-length/2+diameter/2, 0, 1.25*diameter])
    if rod_name == "baseto540":
        return [base_out, config_540_in_1]
    elif rod_name == "baseto430":
        return [base_out, config_430_in_2]
    elif rod_name == "baseto330":
        return [base_out, config_330_in_2]

    # 540
    elif rod_name == "540to540":
        return [config_540_out_2, config_540_in_1]
    elif rod_name == "540to430":
        return [config_540_out_1, config_430_in_1]
    elif rod_name == "540to330":
        return [config_540_out_1, config_330_in_2]

    # 430
    elif rod_name == "430to430":
        return [config_430_out_2, config_430_in_2]
    elif rod_name == "430to330":
        return [config_430_out_1, config_330_in_2]
    
    # 330
    elif rod_name == "330to330":
        return [config_330_out_1, config_330_in_1]
        


# Base and joint
r_540_base = base("assets/540_base/540_base/urdf/540_base.urdf")
r_540_joint = create_revolute_joint("assets/540_joint/540_joint/urdf/540_joint.zip.urdf")
r_430_joint = create_revolute_joint("assets/430_joint/430_joint/urdf/430_joint.urdf")
r_330_joint = create_revolute_joint("assets/330_joint/330_joint/urdf/330_joint.urdf")


baseto540_links = create_i_links(rod_name="baseto540")
baseto430_links = create_i_links(rod_name="baseto430")
baseto330_links = create_i_links(rod_name="baseto330")


J540to540_links = create_i_links(rod_name="540to540")
J540to430_links = create_i_links(rod_name="540to430")
J540to330_links = create_i_links(rod_name="540to330")

J430to430_links = create_i_links(rod_name="430to430")
J430to330_links = create_i_links(rod_name="430to330")

J330to330_links = create_i_links(rod_name="330to330")

# Create database
db = ModulesDB()
db.add(r_540_base)
db.add(r_430_joint)
db.add(r_540_joint)
db.add(r_330_joint)
db = db.union(baseto540_links)
db = db.union(baseto430_links)
db = db.union(baseto330_links)
db = db.union(J540to540_links)
db = db.union(J540to430_links)
db = db.union(J540to330_links)
db = db.union(J430to430_links)
db = db.union(J430to330_links)
db = db.union(J330to330_links)
# viz = db.debug_visualization()

print(r_540_base)

modules = ('540_base', 'baseto540-0.15', '540_joint', '540to430-0.15', '430_joint','430to330-0.15', '330_joint', '330to330-0.15', '330_joint')
B = ModuleAssembly.from_serial_modules(db, modules)

robot = B.to_pin_robot()
# B.to_urdf(write_to=Path("robot6DOF.urdf"))
print("DOF of the robot:",  robot.dof)
viz = B.robot.visualize(coordinate_systems="full")
robot.has_self_collision()

Module Revolute Joint: adaptor_joint, ID: 540_base
DOF of the robot: 5
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7059/static/


False

In [104]:
viz.viewer.jupyter_cell(height=800)

### Things to check
1. The size of the hole on the connector.
2. The base seems rotate a bit at initial state.
### base
1. base to 540_joint
2. base to 430_joint
3. base to 330_joint

### 540_joint
1. 540 to 430
2. 540 to 330
3. 540 to 540

### 430_joint
1. 430 to 430
2. 430 to 330

### 330_joint
N/A